In [2]:
import numpy as np
import cv2
import os
from ultralytics import YOLO

In [3]:
images_dir = "data/images/train/"

In [4]:
model = YOLO("runs/detect/train7/weights/last.pt")
shape_score_dict = {"circle":20,"square":15,"triangle":10,"cross":5}


In [5]:
def calculate_score(results):
    score = 0
    shapes_conversion_dict = results.names
    for obj in map(int,results.cpu().boxes.cls.int()):
        obj = int(obj)
        shape = shapes_conversion_dict[obj]
        score += shape_score_dict[shape]
    return score


In [6]:
from statistics import mode

In [7]:
rtsp = "rtsp://192.168.1.102:8554/unicast"
cap = cv2.VideoCapture(0)
scores = []
while True:
    ret, frame = cap.read()
    results = model(frame,conf=0.5,show=False)
    scores.append(calculate_score(results[0]))
    if len(scores) == 50:
        scores.pop(0)
    score = np.median(scores)
    img = results[0].plot(labels=False)
    cv2.putText(img, f"Score: {score}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow('det',img)
    if cv2.waitKey(1) == ord('d'):
        break
cv2.destroyAllWindows()
cap.release()   


0: 960x1280 (no detections), 156.2ms
Speed: 7.9ms preprocess, 156.2ms inference, 4.7ms postprocess per image at shape (1, 3, 960, 1280)

0: 960x1280 (no detections), 51.0ms
Speed: 7.5ms preprocess, 51.0ms inference, 1.1ms postprocess per image at shape (1, 3, 960, 1280)

0: 960x1280 (no detections), 51.6ms
Speed: 9.0ms preprocess, 51.6ms inference, 0.7ms postprocess per image at shape (1, 3, 960, 1280)

0: 960x1280 (no detections), 45.3ms
Speed: 5.0ms preprocess, 45.3ms inference, 0.9ms postprocess per image at shape (1, 3, 960, 1280)

0: 960x1280 (no detections), 31.8ms
Speed: 6.2ms preprocess, 31.8ms inference, 1.0ms postprocess per image at shape (1, 3, 960, 1280)

0: 960x1280 (no detections), 40.5ms
Speed: 5.4ms preprocess, 40.5ms inference, 0.7ms postprocess per image at shape (1, 3, 960, 1280)

0: 960x1280 (no detections), 45.6ms
Speed: 5.7ms preprocess, 45.6ms inference, 0.8ms postprocess per image at shape (1, 3, 960, 1280)

0: 960x1280 (no detections), 32.0ms
Speed: 5.4ms pre

In [20]:
# don't use this function
def black_detection(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    lower_black = np.array([0, 0, 0], dtype=np.uint8)
    upper_black = np.array([255, 200, 255], dtype=np.uint8)
    black_mask = cv2.inRange(hsv, lower_black, upper_black)
    # black_detected = cv2.bitwise_and(img, img, mask=black_mask)
    return black_mask

In [21]:
for img_name in os.listdir("shapes_pics"):
    img_path = os.path.join("shapes_pics",img_name)
    img = cv2.imread(img_path)
    # crop left half of image
    img = img[65:-60,int(img.shape[1]/2)+150:]
    orig = np.array(img)
    cv2.imwrite(os.path.join("out",img_name),img)
    # img = black_detection(img)
#     cv2.imshow('orig',cv2.resize(orig,None,fx=0.5,fy=0.5))
#     # cv2.imshow('img',cv2.resize(img,None,fx=0.5,fy=0.5))
#     if cv2.waitKey(0)&0xff == ord('d'):
#         break
# cv2.destroyAllWindows()

In [22]:
# capture atef's mobile video stream
cap = cv2.VideoCapture("rtsp://:8080/unicast")
while True:
    ret, frame = cap.read()
    frame = black_detection(frame)
    cv2.imshow('det',frame)
    if cv2.waitKey(1) == ord('d'):
        break
cv2.destroyAllWindows()
cap.release()   

[tcp @ 0x55bfeb822b00] Connection to tcp://10.10.221.34:8080 failed: No route to host
[ERROR:0@1004.016] global cap.cpp:164 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.8.0) /io/opencv/modules/videoio/src/cap_images.cpp:253: error: (-5:Bad argument) CAP_IMAGES: can't find starting number (in the name of file): https://10.10.221.34:8080/unicast in function 'icvExtractPattern'




error: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
